In [2]:
import time 
from torchvision import transforms

import cv2
from PIL import Image

In [3]:
from torchvision import models
import torch 
net = models.quantization.mobilenet_v2(pretrained=True, quantize=True)
net = torch.jit.script(net)

/home/mohamed/Desktop/freelance/face_recognition/opencv_face/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mohamed/Desktop/freelance/face_recognition/opencv_face/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_QuantizedWeights.IMAGENET1K_QNNPACK_V1`. You can also use `weights=MobileNet_V2_QuantizedWeights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/mohamed/Desktop/freelance/face_recognition/opencv_face/lib/python3.10/site-packages/torch/ao/quantization/utils.py:376: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [4]:


cap = cv2.VideoCapture(0)

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

net = models.quantization.mobilenet_v2(pretrained=True, quantize=True)
# jit model to take it from ~20fps to ~30fps
net = torch.jit.script(net)

started = time.time()
last_logged = time.time()
frame_count = 0

with torch.no_grad():
    while True:
        # read frame
        ret, image = cap.read()
        if not ret:
            raise RuntimeError("failed to read frame")

        # convert opencv output from BGR to RGB
        image = image[:, :, [2, 1, 0]]
        permuted = image

        # preprocess
        input_tensor = preprocess(image)

        # create a mini-batch as expected by the model
        input_batch = input_tensor.unsqueeze(0)

        # run model
        output = net(input_batch)
        # do something with output ...

        # log model performance
        frame_count += 1
        now = time.time()
        if now - last_logged > 1:
            print(f"{frame_count / (now-last_logged)} fps")
            last_logged = now
            frame_count = 0

0.7639938353044123 fps
17.685980638473566 fps
19.455268764983796 fps
15.235636236500307 fps
17.31041525752111 fps
18.811983502288832 fps
17.699099144296092 fps
16.26789314590876 fps
18.39306835195641 fps


KeyboardInterrupt: 

In [1]:
import torch
net = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

Using cache found in /home/mohamed/.cache/torch/hub/pytorch_vision_v0.10.0


In [4]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
testset = ImageFolder(root='./att_faces/test', 
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

AttributeError: module 'torch' has no attribute '_six'

In [2]:
PATH = './mobilenet.pth'

net.load_state_dict(torch.load(PATH, weights_only=True))

outputs = net(images)

_, predicted = torch.max(outputs, 1)

# print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
#                               for j in range(4)))

NameError: name 'images' is not defined

# Trying pretrained network for feature exctraction 

In [1]:
import torch 
from torchvision import transforms as T 
import cv2 
from torchvision.models import efficientnet_b0 as effecientnet_model
from  torchvision.models import EfficientNet_B0_Weights
from torch import nn 

In [2]:
class Predictor():

    def __init__(self, n_features=1000):
        super().__init__()
        classes = ['person','not_person']
        n_classes = len(classes)

        model = effecientnet_model(EfficientNet_B0_Weights)    
        n_inputs = model.classifier[1].in_features

        model.classifier[1] = nn.Linear(n_inputs,n_features)
        
        model.cuda()

        self.model = model.eval()


        self.transforms = T.Compose([T.ToTensor(),
                       T.Resize((240,240)),
                       T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                       
                       ])
        


    def predict_cls(self,image): 
        
        image = self.transforms(image.copy())
        image = torch.unsqueeze(image, 0)

        # set the module to evaluation mode
        with torch.no_grad():


            # move data to GPU
            if torch.cuda.is_available():
                image = image.cuda()

            # 1. forward pass: compute predicted outputs by passing inputs to the model
            logits  = self.model(image) # YOUR CODE HERE


        return logits.squeeze()

In [3]:

image1 = cv2.imread("att_faces/webcam/1/image_180.png")
image2 = cv2.imread("att_faces/webcam/1/image_0.png")
pred = Predictor(n_features= 1000)


/home/mohamed/Desktop/freelance/face_recognition/opencv_face/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/mohamed/Desktop/freelance/face_recognition/opencv_face/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:

hep1 = pred.predict_cls(image1)
hep2 = pred.predict_cls(image2)
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

In [5]:
cos(hep1, hep2)

tensor(0.9280, device='cuda:0')

In [6]:
torch.save(hep1, "vec1.pt")

In [7]:
hep3 = torch.load("vec1.pt")

/tmp/ipykernel_5699/173804834.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hep3 = torch.load("vec1.pt")


In [9]:
cos(hep2, hep3)

tensor(0.9280, device='cuda:0')

In [13]:
from datetime import datetime



datetime.datetime

In [17]:
datetime.now().hour

21

In [22]:
import os 
hep = []

In [23]:
max(hep)

ValueError: max() arg is an empty sequence